In [1]:
import os
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma, Vectara
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models.openai import ChatOpenAI
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:


customer_id = '189024573'
corpus_id = '2'
api_key = 'PROVIDE VECTRA API KEY OR ACCESS THROUGH CONFIG FILE'
openai_key = 'PROVIDE OPENAI API KEY OR ACCESS THROUGH CONFIG FILE'
os.environ["VECTARA_CUSTOMER_ID"] = customer_id
os.environ["VECTARA_CORPUS_ID"] = corpus_id
os.environ["VECTARA_API_KEY"] = api_key
os.environ["OPENAI_API_KEY"] = openai_key

In [3]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [89]:
vectara = Vectara()
summary_config = {
    "is_enabled": True, "max_results": 30,
    "response_lang": "en",
    "prompt_name": "Chatbot Q&A"
}
retriever = vectara.as_retriever(
    search_kwargs={"k": 100, "summary_config": summary_config}
)

In [90]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. \
Display the Answer in separate line \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

Question: {question} 

Context: {context} 

Answer:

Explanation:

"""
custom_rag_prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [98]:
result = rag_chain.invoke("What is endocrine diseage?")

In [99]:
print(result)

Endocrine disease usually involves the secretion of too much or not enough hormone. When too much hormone is secreted, it is called hypersecretion. When not enough hormone is secreted, it is called hyposecretion.


In [93]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets .


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mural\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [94]:
tru_recorder = TruChain(rag_chain,
    app_id='Q&A_ChatApplication',
    feedbacks=[f_answer_relevance])

In [100]:
q_list = ["Renal plasma flow equals the blood flow per minute times the what?", 
          "What species do humans belong to?", 
          "Titration is a method to determine what in acids or bases?", 
          "How many different amino acids make up proteins?",
          "What renewable energy source converts energy from the sunlight into electricity?",
          "The bones of the skull are connected by what type of joints?",
          "What is the lowest layer of the atmosphere?",
          "What is the name of the type of plant tissue consisting of undifferentiated cells that can continue to divide and differentiate?",
          "What type of organism is commonly used in preparation of foods such as cheese and yogurt?",
          "Which radio frequency should you listen to if you want less noise?",
          "The fossil record shows that this type of event is followed by the evolution of new species to fill the habitats where old species lived?",
          "Although air can transfer heat rapidly by convection, it is a poor conductor and thus a good what?",
          "What does erosion do to pieces of broken rock?",
          "While the egg is developing, other changes are taking place in the uterus. it develops a thick lining that is full of what?",
          "Collagen fibers, elastic fibers, and reticular fibers comprise what type of tissue?",
          "Surface tension of alveolar fluid, which is mostly water, creates an inward pull of the tissue of what organ?",
          "What distinguishing characteristic of annelid anatomy shows specialization and adaptation?",
          "What organ is subdivided into ascending, descending, transverse and sigmoid parts?",
          "Mushrooms are an example of what type of organism, which includes beneficial and toxic specimens?",
          "Comparing what sequences provides clues to evolution and development?",
          "What is the minimum mass capable of supporting sustained fission called?",
          "Inside the nasal area of the skull, the nasal cavity is divided into halves by the what?",
          "Bacteria can be chemotrophs, which obtain what by breaking down chemical compounds in their environment?",
          "What do we call the cartilaginous structure that surrounds the notochrod?",
          "What is the name for biochemical compounds that consist of one or more chains of small molecules called amino acids?",
          "In a monogamous pairing, a male individual is generally paired with what other type of individual in a sexual relationship?",
          "What property of warm air causes it to rise above cold air?",
          "What organism is characterized by an incomplete digestive system and a single, tentacled opening?",
          "What term is not the same as energy, but means the energy per unit charge?",
          "The simplest class of organic compounds is the what?",
          "The amount of kinetic energy in a moving object depends directly on its mass and what else?"
]

In [103]:
for q in q_list:
    response, tru_record = tru_recorder.with_record(rag_chain.invoke, f"{q}")

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

In [97]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "Fertilization is the union of a sperm and egg, resulting in the formation of what?")

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

In [82]:
tru.get_leaderboard(app_ids=["Q&A_ChatApplication"])

,relevance,latency,total_cost
app_id,,,
Q&A_ChatApplication,1.0,6.0,0.018596


In [83]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.29.93:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>